# Project 1 Notebook for Jonathan Owens
This notebook is to keep track of the various items worked on with a place to collaborate all thoughts, ideas and creations with the team.


## Regional weather site code and elements for the region growing corn, wheat and soybeans
http://http://builder.rcc-acis.org/

- Station ID: KOMA
    - Location: Omaha Eppley Airfield
    - Region: Omaha, Nebrask
- Start date: 2000-10-26
- End date: 2020-10-28
- Elements: pcpn,avgt (percipitation, average temperature)
- Output: csv (comma seperated value)


## Regional Climate Centers (RCC) - Applied Climate Information Systems (ACIS) Documentation
    
- Documentation: http://www.rcc-acis.org/docs_webservices.html

- API URL: http://data.rcc-acis.org/StnData

- API URL ELEMENTS:

    elements = {
        sid=KOMA,
        sdate=20001026,
        edate=20201028,
        elems=pcpn,avgt,
        output=csv
        }

- Missing data values are returned as "M" and traces of precipitation, snowfall or snow depth are returned as "T".


In [1]:
# Libraries
import requests
import pandas as pd
import json
import libs.weather as weather


In [2]:
# Initialize variables
# Station id
station_id = 'KOMA'
# Starting date, furthest back
start_date = '20001026'
# Ending date, most recent
end_date = '20201028'
# Meta data to narrow information returned
meta_data = 'name'
# Data arguments to pull, precipitation, average temperature
elements = 'pcpn,avgt'
# Output to csv or json
output_style = 'json'
# API URL string
rcc_url = f'http://data.rcc-acis.org/StnData?sid={station_id}&sdate={start_date}&edate={end_date}&meta={meta_data}&elems={elements}&output={output_style}'


In [3]:
# Fetch current data
response_data = requests.get(rcc_url)


In [4]:
# Accept json format
data = response_data.json()


In [5]:
# Print json.dumps data
#print(json.dumps(data, indent=4))


In [6]:
# Create dataframe
weather_data =  pd.DataFrame(data['data'])
weather_data.head()


,0,1,2
0,2000-10-26,T,62.0
1,2000-10-27,0.00,54.5
2,2000-10-28,T,56.5
3,2000-10-29,0.68,56.5
4,2000-10-30,0.00,60.0


In [7]:
# Create column headings and assign
weather_data.columns = ['date', 'precipitation', 'average_temperature']
weather_data.head()

,date,precipitation,average_temperature
0,2000-10-26,T,62.0
1,2000-10-27,0.00,54.5
2,2000-10-28,T,56.5
3,2000-10-29,0.68,56.5
4,2000-10-30,0.00,60.0


In [8]:
# Drop rows with values of 'M'
index_rows = weather_data[weather_data.average_temperature == 'M'].index
clean_weather_data = weather_data.drop(index_rows)


In [9]:
# Replace values of 'T' with 0.000000001 to represent a value other than 0 or False
# A measurement of precipitation was detected, but not provided
clean_weather_data['precipitation'] = clean_weather_data['precipitation'].replace('T', 0.000000001)


In [10]:
# Change date values from object to datetime
clean_weather_data['date'] = pd.to_datetime(clean_weather_data['date'])


In [11]:
# Change precipitation values from object to type float
clean_weather_data['precipitation'] = clean_weather_data['precipitation'].astype(float)
# Change average_temperature values from object to type float
clean_weather_data['average_temperature'] = clean_weather_data['average_temperature'].astype(float)


In [12]:
# Set index to date and view clean dataframe
clean_weather_data.set_index('date', inplace=True)
clean_weather_data.head()


,precipitation,average_temperature
date,,
2000-10-26,0.0001,62.0
2000-10-27,0.0000,54.5
2000-10-28,0.0001,56.5
2000-10-29,0.6800,56.5
2000-10-30,0.0000,60.0


In [13]:
def get_precipitation_and_average_temperature(station_id='KOMA', start_date='20001026', end_date='20201026'):
    # API URL string
    rcc_url = f'http://data.rcc-acis.org/StnData?sid={station_id}&sdate={start_date}&edate={end_date}&meta=name&elems=pcpn,avgt&output=json'
    # Fetch current data
    response_data = requests.get(rcc_url)
    # Accept json format
    data = response_data.json()
    # Create dataframe
    weather_data =  pd.DataFrame(data['data'])

    # Create column headings and assign
    weather_data.columns = ['date', 'precipitation', 'average_temperature']

    # Drop rows with values of 'M'
    index_rows = weather_data[weather_data.average_temperature == 'M'].index
    clean_weather_data = weather_data.drop(index_rows)
    # Replace values of 'T' with 0.000000001 to represent a value other than 0 or False
    # A measurement of precipitation was detected, but not provided
    clean_weather_data['precipitation'] = clean_weather_data['precipitation'].replace('T', 0.000000001)

    # Change date values from object to datetime
    clean_weather_data['date'] = pd.to_datetime(clean_weather_data['date'])
    # Change precipitation values from object to type float
    clean_weather_data['precipitation'] = clean_weather_data['precipitation'].astype(float)
    # Change average_temperature values from object to type float
    clean_weather_data['average_temperature'] = clean_weather_data['average_temperature'].astype(float)

    # Set date index
    clean_weather_data.set_index('date', inplace=True)

    # Return clean weather data
    return clean_weather_data


In [14]:
# Testing weather function above
df = get_precipitation_and_average_temperature('KOMA', 20201015, 20201028)
df.head()


,precipitation,average_temperature
date,,
2020-10-15,0.0000,47.5
2020-10-16,0.0000,46.5
2020-10-17,0.0001,56.0
2020-10-18,0.2300,38.5
2020-10-19,0.0100,40.0


In [15]:
# Testing weather library and function
returned_data = weather.get_precipitation_and_average_temperature()
returned_data.head()


,precipitation,average_temperature
date,,
2000-10-26,0.0001,62.0
2000-10-27,0.0000,54.5
2000-10-28,0.0001,56.5
2000-10-29,0.6800,56.5
2000-10-30,0.0000,60.0


In [16]:
# Build dataframe of three stations, 'KOMA', 'KCDR', 'KMCK'
df = weather.get_precipitation_and_average_temperature('KOMA', '20201020', '20201028')
df2 = weather.get_precipitation_and_average_temperature('KCDR', '20201020', '20201028')
df3 = weather.get_precipitation_and_average_temperature('KMCK', '20201020', '20201028')


In [18]:
# Average the precipitations and temperatures
print((df + df2 + df3) / 3)


            precipitation  average_temperature
date                                          
2020-10-20       0.006700            44.166667
2020-10-21       0.006700            42.500000
2020-10-22       0.026733            36.000000
2020-10-23       0.000033            29.000000
2020-10-24       0.000033            28.333333
2020-10-25       0.023400            22.333333
2020-10-26       0.043333            16.000000
2020-10-27       0.000000            21.833333
2020-10-28       0.000000            37.666667


In [19]:
# Dictionary of states and station codes
# Can create a connection to a sql db in the future to store this information for data growth
state_station_codes = {
    'NE': ['KOMA', 'KCDR', 'KMCK'],
    'IA': ['KCID', 'KDSM', 'KALO'],
    'IL': ['KORD', 'KMDW', 'KSPI']
    }


In [17]:
def from_state(state_name='NE', start_date='20201001', end_date='20201005', return_format='df'):
    state_df = {}
    # Check for valid arguments passed in by user
    if state_name == 'NE' or 'IA' or 'IL':
        # Get station ids for selected state
        list_of_stations = state_station_codes[state_name]

        # Get station data for each station
        for station in list_of_stations:
            list_of_station_dataframes = weather.get_precipitation_and_average_temperature(station, start_date, end_date)
        # Add all station data into one dataframe
        for station_data in list_of_station_dataframes:
            print(station_data)

        # Calculate average of combine data values
        
        
        # Return state dateframe
        if return_format == 'df':
            return state_df

        # Output to csv file
        elif return_format == 'csv':
            # Create output path and write data to csv
            csv_output_path = Path('./clean_data/state_historical_weather.csv')
            state_df.to_csv(csv_output_path)
        else:
            # Return message that return format not found
            return 'Return format specified not found.  Pass in df or csv as a string.'
    else: 
        # Return message that state not found
        return 'State not found.  Pass in NE, IA or IL as a string.'


In [23]:
response = from_state('NE', '20201001', '20201005','df')
response

precipitation
average_temperature


{}

# Notes to ask the group and for us to think about:
1. Is there an existing weather.py library on the internet
2. README.md, offer a template that I use to start
